In [1]:
!ls '/kaggle/input/digital-knee-x-ray-images/Digital Knee X-ray Images/Digital Knee X-ray Images/Knee X-ray Images/MedicalExpert-I/MedicalExpert-I/'

0Normal  1Doubtful  2Mild  3Moderate  4Severe


In [2]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer='/kaggle/input/digital-knee-x-ray-images/Digital Knee X-ray Images.csv')
df

,Parent Directory,Subdirectory,File Count
0,MedicalExpert-I,0Normal,514
1,MedicalExpert-I,1Doubtful,477
2,MedicalExpert-I,2Mild,232
3,MedicalExpert-I,3Moderate,221
4,MedicalExpert-I,4Severe,206
5,MedicalExpert-II,0Normal,503
6,MedicalExpert-II,1Doubtful,488
7,MedicalExpert-II,2Mild,232
8,MedicalExpert-II,3Moderate,221
9,MedicalExpert-II,4Severe,206


In [3]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np


DEVICE = torch.device('cpu')
OUTPUT_SIZE = 2048

model = models.resnext50_32x4d(weights=models.ResNeXt50_32X4D_Weights.IMAGENET1K_V2)

extraction_layer = model._modules.get('avgpool')
model.to(DEVICE)
model.eval()

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vec(arg, model, extraction_layer):
    image = normalize(to_tensor(scaler(arg))).unsqueeze(0).to(DEVICE)
    result = torch.zeros(1, OUTPUT_SIZE, 1, 1)
    def copy_data(m, i, o):
        result.copy_(o.data)
    hooked = extraction_layer.register_forward_hook(copy_data)
    with torch.no_grad():
        model(image)
    hooked.remove()
    return result

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth
100%|██████████| 95.8M/95.8M [00:29<00:00, 3.46MB/s]


In [4]:
import arrow
import base64
import pandas as pd
from glob import iglob
from io import BytesIO
from os.path import basename
from PIL import Image

THUMBNAIL_SIZE = (64, 64)
ME1 = '/kaggle/input/digital-knee-x-ray-images/Digital Knee X-ray Images/Digital Knee X-ray Images/Knee X-ray Images/MedicalExpert-I/MedicalExpert-I'
ME2 = '/kaggle/input/digital-knee-x-ray-images/Digital Knee X-ray Images/Digital Knee X-ray Images/Knee X-ray Images/MedicalExpert-II/MedicalExpert-II'

def embed(model, filename: str):
    with Image.open(fp=filename, mode='r') as image:
        return get_vec(arg=image.convert('RGB'), model=model, extraction_layer=extraction_layer).numpy().reshape(OUTPUT_SIZE,)


# https://stackoverflow.com/a/952952
def flatten(arg):
    return [x for xs in arg for x in xs]

def png(filename: str) -> str:
    with Image.open(fp=filename, mode='r') as image:
        buffer = BytesIO()
        # our images are pretty big; let's shrink the hover images to thumbnail size
        image.resize(size=THUMBNAIL_SIZE).convert('RGB').save(buffer, format='png')
        return 'data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode()

def get_picture_from_glob(arg: str, tag: str,) -> list:
    time_get = arrow.now()
    result = [pd.Series(data=[tag, basename(input_file), embed(model=model, filename=input_file), png(filename=input_file)],
                        index=['tag', 'name', 'value', 'png'])
        for index, input_file in enumerate(list(iglob(pathname=arg))) if input_file.endswith('.png')]
    print('encoded {} rows of {}  in {}'.format(len(result), tag, arrow.now() - time_get))
    return result

time_start = arrow.now()
me1_dict = {basename(folder) : folder + '/*.*' for folder in iglob(ME1 + '/*')}
me1_df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key) for key, value in me1_dict.items()]))
me1_df['expert'] = 1
me2_dict = {basename(folder) : folder + '/*.*' for folder in iglob(ME2 + '/*')}
me2_df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key) for key, value in me2_dict.items()]))
me2_df['expert'] = 2
df = pd.concat(axis='index', objs=[me1_df, me2_df])
print('done in {}'.format(arrow.now() - time_start))

encoded 477 rows of 1Doubtful  in 0:00:51.078200
encoded 221 rows of 3Moderate  in 0:00:23.874753
encoded 206 rows of 4Severe  in 0:00:22.101054
encoded 232 rows of 2Mild  in 0:00:24.709120
encoded 514 rows of 0Normal  in 0:00:54.668069
encoded 488 rows of 1Doubtful  in 0:00:53.521036
encoded 221 rows of 3Moderate  in 0:00:23.134874
encoded 206 rows of 4Severe  in 0:00:22.229648
encoded 232 rows of 2Mild  in 0:00:25.351514
encoded 503 rows of 0Normal  in 0:00:54.566217
done in 0:05:55.382671
